In [1]:
def CountDown(n):
    while n > 0:
        n -= 1
        
%time CountDown(100000000)

Wall time: 5.08 s


#### 多线程

In [2]:
from threading import Thread

def multi_thread():
    n = 100000000
    t1 = Thread(target=CountDown,args=[n // 2])
    t2 = Thread(target=CountDown,args=[n // 2])
    t1.start()
    t2.start()
    t1.join()
    t2.join()

%time multi_thread()

Wall time: 4.98 s


Python的线程,底层是操作系统的线程,在Linux中是Pthread(全称是:POSIX Thread)<br/>
在Windows系统里是Windows Thread.

Python最流行的解释器是CPython,GIL是CPython的一个技术术语,意思是全局解释器锁,本质上是类似操作系统的Mutex,每一个Python线程,在CPython解释器中执行时,都会先锁住自己的线程,阻止别的线程执行

CPython使用引用计数来管理内存,当引用计数只有0时,则会自动释放内存

In [3]:
import sys
a = []
b = a
sys.getrefcount(a)

3

如果由两个Python线程同时引用了a,就会造成引用计数race condition,引用计数可能最终只增加1,这样就会造成内存被污染,因为第一个线程结束时,会把引用计数减少1 , 这时可能达到条件释放内存,当第二个线程再试图访问a时,就找不到有效的内存了.
所以 , CPython引进GIL其实主要就是这么两个原因:

* 一是设计者为了规避类似于内存管理这样的复杂的竞争风险问题(race condition)
* 二是因为CPython大量使用c语言,但大部分c语言库都不是原生线程安全的

CPython中还有另一个机制,叫做check_interval,意思是CPython解释器回去轮询检查线程GIL的锁住情况,每隔一段时间,Python解释器就会强制当前线程去释放GIL,这样别的线程才有执行机会,用宋小宝的话说就是"雨露均沾"

In [14]:
import threading

n = 0

def foo():
    global n
    n += 1

threads = []
for i in range(100):
    t = threading.Thread(target=foo)
    threads.append(t)
    
for t in threads:
    t.start()
    
for t in threads:
    t.join()
    
print(n)

100


In [15]:
import dis
dis.dis(foo)

  7           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [17]:
n = 0
lock = threading.Lock()
def foo():
    global n
    with lock:
        n += 1

### 如何绕过GIL

如果你的代码不需要CPython解释器执行,就不再受GIL的限制,例如NumPy的矩阵运算,大部分是c实现